In [1]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../fuzzy-tools') # or just install the module

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np

## Train-Val Split
train_kwargs = {
    'root':'../data/',
    'train':True,
    'download':True,
    'transform':transforms.Compose([transforms.ToTensor()]),
}
val_kwargs = {
    'root':'../data/',
    'train':False,
    'download':True,
    'transform':transforms.Compose([transforms.ToTensor()]),
}
train_cifar10 = datasets.CIFAR10(**train_kwargs)
val_cifar10 = datasets.CIFAR10(**val_kwargs)

ImportError: FloatProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
%load_ext autoreload
%autoreload 2
from datasets import MyDataset
import numpy as np
from fuzzytorch.utils import print_tdict

## Batch Sizes
train_batch_size = 256
val_batch_size = train_batch_size

train_dataset_mnist = MyDataset(train_cifar10.data, train_cifar10.targets, uses_da=True)
val_dataset_mnist = MyDataset(val_cifar10.data, val_cifar10.targets)
val_dataset_mnist.set_norm_values(*train_dataset_mnist.get_norm_values())
print_tdict(train_dataset_mnist[0])

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt

## DataLoaders
train_loader_mnist = torch.utils.data.DataLoader(train_dataset_mnist, batch_size=train_batch_size, shuffle=True)
val_loader_mnist = torch.utils.data.DataLoader(val_dataset_mnist, batch_size=val_batch_size)

# print example
for k,tensor_dict in enumerate(train_loader_mnist):
#for k,(data, target) in enumerate(val_loader_mnist):
    print_tdict(tensor_dict)
    ind = 37
    data = tensor_dict['input']['x']
    target = tensor_dict['target']['y']
    img = data[ind].permute(1,2,0).numpy()
    plt.imshow(img)
    plt.title(f'target: {target[ind]}')
    break
    
print(len(train_loader_mnist))
print(len(val_loader_mnist))

# model parameters

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
from fuzzytools.datascience.grid_search import GDIter, GridSeacher
from baseline_models import MLPClassifier, CNN2DClassifier

mdl_params = {
    #'mdl_class':MLPClassifier,
    'mdl_class':CNN2DClassifier,
    'mdl_kwargs':{
        'dropout':0.5,
        #'dropout':0.0,
        'cnn_features':[16, 32, 64],
        #'cnn_features':[16, 32],
        'uses_mlp_classifier':True,
        #'uses_mlp_classifier':False,
    },
}
gs = GridSeacher(mdl_params)
print(gs)

# training

In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID' # see issue #152
os.environ['CUDA_VISIBLE_DEVICES'] = '' # CPU

### LOSS
from fuzzytorch.losses import XEntropy

loss_kwargs = {
    'model_output_is_with_softmax':False,
    'target_is_onehot':False,
}
loss = XEntropy('xentropy', **loss_kwargs)

### METRICS
from fuzzytorch.metrics import DummyAccuracy, Accuracy
metrics = [
    Accuracy('accuracy', balanced=False, **loss_kwargs),
    Accuracy('b-accuracy', balanced=True, **loss_kwargs),
    DummyAccuracy('dummy-accuracy', **loss_kwargs),
]

### GET MODEL
model = mdl_params['mdl_class'](**mdl_params['mdl_kwargs'])

### OPTIMIZER
import torch.optim as optims
from fuzzytorch.optimizers import LossOptimizer

optimizer_kwargs = {
    'opt_kwargs':{
        'lr':1e-3,
    },
    'decay_kwargs':{
        'lr':0.9,
    }
}
optimizer = LossOptimizer(model, optims.Adam, **optimizer_kwargs)

### MONITORS
from fuzzytools.prints import print_bar
from fuzzytorch.handlers import ModelTrainHandler
from fuzzytorch.monitors import LossMonitor
from fuzzytorch import C_

monitor_config = {
    'val_epoch_counter_duration':0, # every k epochs check
    #'val_epoch_counter_duration':2, # every k epochs check
    #'earlystop_epoch_duration':1e2,
    #'save_mode':C_.SM_NO_SAVE,
    #'save_mode':C_.SM_ALL,
    #'save_mode':C_.SM_ONLY_ALL,
    #'save_mode':C_.SM_ONLY_INF_METRIC,
    'save_mode':C_.SM_ONLY_INF_LOSS,
    #'save_mode':C_.SM_ONLY_SUP_METRIC,
}
loss_monitors = LossMonitor(loss, optimizer, metrics, **monitor_config)

### TRAIN
mtrain_config = {
    'id':0,
    'epochs_max':1e3,
    'save_rootdir':'../save',
}
model_train_handler = ModelTrainHandler(model, loss_monitors, **mtrain_config)
model_train_handler.build_gpu(gpu_index=None)
print(model_train_handler)
model_train_handler.fit_loader(train_loader_mnist, val_loader_mnist)

In [ ]:
%load_ext autoreload
%autoreload 2
# loss_df opt_df loss_df_epoch metrics_df_epoch
loss_monitors.get_time_util_convergence()


In [ ]:
# loss_df opt_df loss_df_epoch metrics_df_epoch
loss_monitors.get_save_dict()['opt_df']

In [ ]:
# loss_df opt_df loss_df_epoch metrics_df_epoch
loss_monitors.get_save_dict()['loss_df_epoch']

In [ ]:
# loss_df opt_df loss_df_epoch metrics_df_epoch
loss_monitors.get_save_dict()['metrics_df_epoch']

In [ ]:
%load_ext autoreload
%autoreload 2
from fuzzytools.counters import Counter

d = {
'val_epoch_counter_duration':1,
'earlystop_epoch_duration':5,
}
c = Counter(d)
for _ in range(50):
    print(c, c.check('earlystop_epoch_duration'))
    c.update()

In [ ]:
%load_ext autoreload
%autoreload 2
import flamingChoripan.tinyFlame.plots as tfplots

### training plots
fig, ax = tfplots.plot_trainloss(train_handler)
fig, ax = tfplots.plot_evaluation_loss(train_handler)
fig, ax = tfplots.plot_evaluation_metrics(train_handler)
#fig, ax = tfplots.plot_optimizer(train_handler, save_dir=mtrain_config['images_save_dir'])
plt.show()
        

# prediction and CM